In [2]:
pip install ta 

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29411 sha256=6fb8b446b8a493fcc04f2b8912acf7c4c4541b3049f7c91da9846eb895b35ede
  Stored in directory: /Users/ayushsingh/Library/Caches/pip/wheels/5c/a1/5f/c6b85a7d9452057be4ce68a8e45d77ba34234a6d46581777c6
Successfully built ta
Note: you may need to restart the kernel to use updated packages.


In [17]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def etl_stock_data(file_path):
    # Load dataset
    df = pd.read_csv('df_final.csv')
    
    # Convert date column to datetime
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values(by=['ticker', 'date'])
    
    # Remove unnecessary columns
    df = df[['ticker', 'date', 'open', 'high', 'low', 'close', 'adj. close', 'volume']]
    
    # Feature Engineering
    df['daily_return'] = df.groupby('ticker')['adj. close'].pct_change()
    df['volatility'] = df.groupby('ticker')['daily_return'].rolling(window=5).std().reset_index(level=0, drop=True)
    df['5_day_ma'] = df.groupby('ticker')['adj. close'].rolling(window=5).mean().reset_index(level=0, drop=True)
    df['10_day_ma'] = df.groupby('ticker')['adj. close'].rolling(window=10).mean().reset_index(level=0, drop=True)
    df['20_day_ma'] = df.groupby('ticker')['adj. close'].rolling(window=20).mean().reset_index(level=0, drop=True)
    
    # Shift close price to serve as next day's prediction target
    df['next_day_close'] = df.groupby('ticker')['adj. close'].shift(-1)
    
    # Drop rows with NaN values (caused by rolling operations)
    df = df.dropna()
    
    # Normalize numeric features
    scaler = MinMaxScaler()
    numerical_cols = ['open', 'high', 'low', 'close', 'adj. close', 'volume', 'daily_return', 'volatility', '5_day_ma', '10_day_ma', '20_day_ma']
    df[numerical_cols] = scaler.fit_transform(df[numerical_cols])
    
    return df

# Example usage:
file_path = "df_final.csv"
cleaned_df = etl_stock_data(file_path)

# Save processed data
cleaned_df.to_csv("/mnt/data/processed_stock_data.csv", index=False)


/opt/anaconda3/lib/python3.12/site-packages/sklearn/utils/_array_api.py:472: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/utils/_array_api.py:489: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))


ETL Process Completed. Data saved at: transformed_stock_data.csv


f"/Users/ayushsingh/Desktop/MBD/Python 2/Group Assignment/stock_predictor_app/models/{ticker}_price_model.joblib")